In [1]:
import os
import sqlite3

In [2]:
'''
一共開 3 個 DB
1. 存 NER 結果
2. 存 POS 結果
3. 存 RE 結果: 含網址、Email、Username、(行動、市內)電話、IP
'''

con_NER = sqlite3.connect("JudicialAnalysis_NER.db")
cur_NER = con_NER.cursor()

con_POS = sqlite3.connect("JudicialAnalysis_POS.db")
cur_POS = con_POS.cursor()

con_RE = sqlite3.connect("JudicialAnalysis_RE.db")
cur_RE = con_RE.cursor()


In [7]:
# 統計所有的 Crime
crimes = dict()
check_res = con_RE.execute(f"SELECT keywords FROM results")
temp = check_res.fetchone()
while temp:
    keys = temp[0].strip().split(',')
    for key in keys:
        crimes.setdefault(key, 0)
        crimes[key] += 1
    temp = check_res.fetchone()
        

In [10]:
crimes_sort = sorted(crimes.items(), key =lambda x : x[1],reverse=True)
crimes_sort[:20]

[('Z000000000', 7825),
 ('毒品危害防制條例', 5466),
 ('公共危險', 4596),
 ('詐欺', 4202),
 ('洗錢防制法', 3686),
 ('LINE', 3655),
 ('竊盜', 3470),
 ('過失傷害', 2755),
 ('被害人告訴人', 2252),
 ('施用毒品', 1822),
 ('於本', 1557),
 ('傷害', 1402),
 ('SIM', 1046),
 ('就本', 817),
 ('ATM', 688),
 ('偽造文書', 645),
 ('妨害自由', 641),
 ('前施用毒品', 632),
 ('IMEI', 567),
 ('為本', 487)]

In [12]:
# 找出所有與詐欺有關的裁判書

fraud_jids = set()
check_res = con_RE.execute(f"SELECT jid, keywords FROM results")
temp = check_res.fetchone()
while temp:
    raw = temp[1].strip()
    jid = temp[0].strip()
    if '詐欺' in raw:
        fraud_jids.add(jid)
    temp = check_res.fetchone()
fraud_jids = list(fraud_jids)
fraud_jids.sort()
print(f'>>總共:{len(fraud_jids)}')
fraud_jids[:20]

>>總共:5286


['CHDM,110,易,451,20230131,1',
 'CHDM,110,訴,1011,20230131,2',
 'CHDM,110,訴,1064,20230112,2',
 'CHDM,110,訴,602,20230203,1',
 'CHDM,110,訴,658,20230221,1',
 'CHDM,110,訴,879,20230203,1',
 'CHDM,110,訴,936,20230109,1',
 'CHDM,110,附民,151,20230131,1',
 'CHDM,110,附民,296,20230109,1',
 'CHDM,110,附民,305,20230109,1',
 'CHDM,110,附民,306,20230109,1',
 'CHDM,110,附民,324,20230109,1',
 'CHDM,110,附民,325,20230109,1',
 'CHDM,110,附民,352,20230109,1',
 'CHDM,111,原訴,18,20230215,2',
 'CHDM,111,原訴,25,20230111,1',
 'CHDM,111,原訴,34,20230131,1',
 'CHDM,111,原附民,14,20230119,1',
 'CHDM,111,撤緩,99,20230210,1',
 'CHDM,111,易,1091,20230110,1']

In [28]:
'''
找出所有與詐欺有關的各類關鍵字
'''

con_Fraud = sqlite3.connect("JudicialAnalysis_Fraud.db")
cur_Fraud = con_Fraud.cursor()

# 建立新的 DB
new = 0
if new:
    cur_Fraud.execute("CREATE TABLE results(keyword_type, keyword, frequency)")
    con_Fraud.commit()
    con_Fraud.close()

In [19]:
# 開始整理
from IPython.display import clear_output

fraud_key_dicts = dict()
count = 0
total = len(fraud_jids)
for idx, jid in enumerate(fraud_jids):
    clear_output(wait=True)
    if (idx % 100) == 0 or idx == (total - 1):
        print(f'[目前進度] {idx+1}/{total}')
        print(f'count = {count}')
        
    # 從 JudicialAnalysis_RE.db 開始
    check_res = con_RE.execute(f"SELECT jid, date,  keyword_type, keywords FROM results WHERE jid='{jid}'")
    rows = check_res.fetchall()
    for row in rows:
        date = row[1].strip()
        k_type = row[2].strip()
        key_words = row[3].strip().split(',')
        
        fraud_key_dicts.setdefault(k_type, dict())
        
        for temp_key in key_words:
            fraud_key_dicts[k_type].setdefault(temp_key, 0)
            fraud_key_dicts[k_type][temp_key] += 1
            count += 1
            

    # 然後 JudicialAnalysis_NER.db 開始
    check_res = con_NER.execute(f"SELECT jid, date,  keyword_type, keywords FROM results WHERE jid='{jid}'")
    rows = check_res.fetchall()
    for row in rows:
        date = row[1].strip()
        k_type = row[2].strip()
        key_words = row[3].strip().split(',')
        
        fraud_key_dicts.setdefault(k_type, dict())
        
        for temp_key in key_words:
            fraud_key_dicts[k_type].setdefault(temp_key, 0)
            fraud_key_dicts[k_type][temp_key] += 1
            count += 1
    
    # 然後 JudicialAnalysis_POS.db 開始
    check_res = con_POS.execute(f"SELECT jid, date,  keyword_type, keywords FROM results WHERE jid='{jid}'")
    rows = check_res.fetchall()
    for row in rows:
        date = row[1].strip()
        k_type = row[2].strip()
        key_words = row[3].strip().split(',')
        
        fraud_key_dicts.setdefault(k_type, dict())
        
        for temp_key in key_words:
            fraud_key_dicts[k_type].setdefault(temp_key, 0)
            fraud_key_dicts[k_type][temp_key] += 1
            count += 1

[目前進度] 5286/5286
count = 3263274


In [29]:
# 將關鍵字存下來
# keyword_type, keyword, frequency
for k_type, keywords in fraud_key_dicts.items():
    keywords_list = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
    for (kl, freq) in keywords_list:
        to_db_content = "INSERT INTO results VALUES(?, ?, ?)" # ? 是一個參數占位符，代表將要插入的值。
        cur_Fraud.execute(to_db_content, (k_type, kl, freq))
        con_Fraud.commit()
con_Fraud.close()

KeyboardInterrupt: 

In [ ]:
con_Fraud.close()

In [30]:
# 將關鍵字按類型存成 xlsx
import pandas as pd

columns = ['關鍵字詞', '出現次數']

for k_type, keywords in fraud_key_dicts.items():
    rows = []
    
    keywords_list = sorted(keywords.items(), key=lambda x:x[1], reverse=True)
    for (kl, freq) in keywords_list:
        row = [kl, freq]
        rows.append(row)
        
    
    df = pd.DataFrame(rows, columns=columns)
    
    xlsx_file = f'fraud({k_type}).xlsx'
    df.to_excel(xlsx_file)